### Import Modules

In [1]:
import fiona
from shapely.geometry import shape, Point, asShape
from shapely import speedups
speedups.enable()
import datetime
import pandas

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### Set paths

In [2]:
points = "/users/danielcorcoran/desktop/test_coordinates_10K.csv"
shapefile = "/users/danielcorcoran/desktop/github_repos/python_nb_data_spatial/data/LGA_ABS_16/LGA_2016_AUST.shp"

### Import points

In [3]:
data = pandas.read_csv("/users/danielcorcoran/desktop/test_coordinates_10K.csv")

In [4]:
print(data.shape)

(10000, 3)


In [5]:
data.columns

Index(['longitude', 'latitude', 'id'], dtype='object')

In [6]:
long_name = 'longitude'
lat_name = 'latitude'

In [7]:
all_polygons = [polygon for polygon in fiona.open(shapefile)]

In [8]:
vic_polygons = []

for polygon in all_polygons:
    if polygon['properties']['STE_NAME16'] == 'Victoria':
        vic_polygons.append(polygon)

### Build spatial tree

In [9]:
from rtree import index

rtree_index = index.Index()

In [10]:
for position, polygon in enumerate(vic_polygons):
    try:
        rtree_index.insert(position, shape(polygon['geometry']).bounds)
    except:
        print("Error at position", position)

Error at position 80
Error at position 81


### Process  (with apply)

In [11]:
def return_properties(point):
    for index in rtree_index.intersection(point.coords[0]):
        if point.within(shape(vic_polygons[index]['geometry'])):
            return vic_polygons[index]['properties']
            break

In [12]:
data['point'] = (list(zip(data[long_name], data[lat_name])))

In [13]:
data["shapely_point"] = data['point'].apply(Point)

In [14]:
data.head()

,longitude,latitude,id,point,shapely_point
0,144.200311,-36.814039,unique_0,"(144.20031142468736, -36.814038954850226)",POINT (144.2003114246874 -36.81403895485023)
1,144.296159,-32.650504,unique_1,"(144.2961591274601, -32.650504069735746)",POINT (144.2961591274601 -32.65050406973575)
2,147.628548,-38.126470,unique_2,"(147.62854800221467, -38.126469641893785)",POINT (147.6285480022147 -38.12646964189378)
3,144.437691,-37.149164,unique_3,"(144.43769068875181, -37.14916386980843)",POINT (144.4376906887518 -37.14916386980843)
4,147.175428,-37.748786,unique_4,"(147.17542810183818, -37.74878585769388)",POINT (147.1754281018382 -37.74878585769388)


In [15]:
data.head()

,longitude,latitude,id,point,shapely_point
0,144.200311,-36.814039,unique_0,"(144.20031142468736, -36.814038954850226)",POINT (144.2003114246874 -36.81403895485023)
1,144.296159,-32.650504,unique_1,"(144.2961591274601, -32.650504069735746)",POINT (144.2961591274601 -32.65050406973575)
2,147.628548,-38.126470,unique_2,"(147.62854800221467, -38.126469641893785)",POINT (147.6285480022147 -38.12646964189378)
3,144.437691,-37.149164,unique_3,"(144.43769068875181, -37.14916386980843)",POINT (144.4376906887518 -37.14916386980843)
4,147.175428,-37.748786,unique_4,"(147.17542810183818, -37.74878585769388)",POINT (147.1754281018382 -37.74878585769388)


In [16]:
start = datetime.datetime.now()

data['results'] = data['shapely_point'].apply(return_properties)
data = pandas.concat([data, data['results'].apply(pandas.Series)], axis = 1)
data.drop(['shapely_point', 'results', 'point'], axis = 1, inplace = True)

print("\nFinished in {}.".format(datetime.datetime.now() - start))


Finished in 0:00:24.963884.


In [17]:
print(data.head())

    longitude   latitude        id                                      point  \
0  144.200311 -36.814039  unique_0  (144.20031142468736, -36.814038954850226)   
1  144.296159 -32.650504  unique_1   (144.2961591274601, -32.650504069735746)   
2  147.628548 -38.126470  unique_2  (147.62854800221467, -38.126469641893785)   
3  144.437691 -37.149164  unique_3   (144.43769068875181, -37.14916386980843)   
4  147.175428 -37.748786  unique_4   (147.17542810183818, -37.74878585769388)   

  LGA_CODE16           LGA_NAME16 STE_CODE16 STE_NAME16  AREASQKM16  
0      22620  Greater Bendigo (C)          2   Victoria   2999.9721  
1        NaN                  NaN        NaN        NaN         NaN  
2        NaN                  NaN        NaN        NaN         NaN  
3      25430  Mount Alexander (S)          2   Victoria   1529.6184  
4      26810       Wellington (S)          2   Victoria  10817.3346  


### Export

In [18]:
data.to_csv('/users/danielcorcoran/desktop/test_points.csv', index = False)

### Process (without apply)
- Note this method is slower than the method above